# xESMF Regridding:  Curvilinear HRRR  => Rectilinear

##### To Do:
* check wind rotation 
* check units on all variables
* determine what other attributes ROMS *requires* in a rectilinear forcing file

In [ ]:
import xarray as xr
import xesmf as xe
import numpy as np
import fsspec
import hvplot.xarray
import geoviews as gv
from matplotlib import path 

HRRR "best time series" for 2019, constructed from archived HRRR forecast data using forecast hour 1 (tau=1)

In [ ]:
url = 's3://esip-qhub/noaa/HRRR/2019'
ds = xr.open_zarr(fsspec.get_mapper(url, requester_pays=True), consolidated=True)
ds = ds.assign_coords(longitude=(((ds.longitude + 180) % 360) - 180))

In [ ]:
ds

In [ ]:
#ds = xr.open_dataset('hrrr_de_bay_2019.nc')

In [ ]:
#zarr_url = 's3://esip-qhub/usgs/de_bay/hrrr_de_bay.zarr'
#ds = xr.open_zarr(fsspec.get_mapper(zarr_url, requester_pays=True, profile='esip-qhub'), consolidated=True)

In [ ]:
ds.data_vars

#### xESMF looks for coordinate variables named 'lon' and 'lat' and ROMS expects specific data variable names 

In [ ]:
ds  = ds.unify_chunks()

In [ ]:
ds = ds.rename({'longitude':'lon', 
                'latitude':'lat',
                'TMP_2maboveground':'Tair',
                'RH_2maboveground':'Qair',
                'UGRD_10maboveground':'ugrid',
                'VGRD_10maboveground':'vgrid'})

In [ ]:
ds['Tair'] = ds['Tair'] - 273.15    # Kelvin to Celsius

Rotate winds using formula from https://rapidrefresh.noaa.gov/faq/HRRR.faq.html

In [ ]:
rotcon_p = 0.622515
lon_xx_p = -97.5
lat_tan_p = 38.8

angle2 = rotcon_p * (ds['lon'] - lon_xx_p)*np.pi/180.
sinx2 = np.sin(angle2)
cosx2 = np.cos(angle2)

ds['Uwind'] =  cosx2*ds['ugrid'] + sinx2*ds['vgrid']
ds['Vwind'] = -sinx2*ds['ugrid'] + cosx2*ds['vgrid']


In [ ]:
da = ds['Uwind'].sel(time='2019-03-25 00:00', method='nearest')
da.hvplot.quadmesh(x='lon', y='lat', geo=True, rasterize=True, cmap='turbo', tiles='OSM')

##### Before we regrid to rectilinear, let's subset a region that covers our ROM area of interest.  Becuase lon,lat are 2D arrays, we can't just use xarray to slice these coordinate variables.  So we have a routine that finds the i,j locations of a specified bounding box, and then slice on those.

In [ ]:
bbox = [-76.63290610753754, -73.55671530588432, 37.57888442021855, 41.225532965406224]

In [ ]:
def bbox2ij(lon,lat,bbox=[-160., -155., 18., 23.]):
    """Return indices for i,j that will completely cover the specified bounding box.     
    i0,i1,j0,j1 = bbox2ij(lon,lat,bbox)
    lon,lat = 2D arrays that are the target of the subset
    bbox = list containing the bounding box: [lon_min, lon_max, lat_min, lat_max]

    Example
    -------  
    >>> i0,i1,j0,j1 = bbox2ij(lon_rho,[-71, -63., 39., 46])
    >>> h_subset = nc.variables['h'][j0:j1,i0:i1]       
    """
    bbox=np.array(bbox)
    mypath=np.array([bbox[[0,1,1,0]],bbox[[2,2,3,3]]]).T
    p = path.Path(mypath)
    points = np.vstack((lon.ravel(),lat.ravel())).T   
    n,m = np.shape(lon)
    inside = p.contains_points(points).reshape((n,m))
    ii,jj = np.meshgrid(range(m),range(n))
    return min(ii[inside]),max(ii[inside]),min(jj[inside]),max(jj[inside])

In [ ]:
i0,i1,j0,j1 = bbox2ij(ds['lon'].values, ds['lat'].values, bbox=bbox)
print(i0,i1,j0,j1)

In [ ]:
ds_subset = ds.isel(x=slice(i0,i1), y=slice(j0,j1))

In [ ]:
ds_subset

In [ ]:
ds.data_vars

In [ ]:
print(ds.Tair)

In [ ]:
da = ds_subset.Tair.sel(time='2019-03-25 00:00', method='nearest')
viz = da.hvplot.quadmesh(x='lon', y='lat', geo=True, rasterize=True, cmap='turbo')
base = gv.tile_sources.OSM
base * viz.opts(alpha=0.5)

In [ ]:
ds_subset = ds_subset.chunk({'x':-1, 'y':-1, 'time':1})

#### Finally, let's specify the region we actually want to create a rectlinear grid for, and the spatial resolution

In [ ]:
bbox = [-76.8, -73.7, 37.7, 40.38]
dx = dy = 3./111.    # 3km grid

In [ ]:
%%time
ds_out = xr.Dataset({'lon': (['lon'], np.arange(bbox[0], bbox[1], dx)),
                     'lat': (['lat'], np.arange(bbox[2], bbox[3], dy))})

regridder = xe.Regridder(ds_subset, ds_out, 'bilinear')
regridder

In [ ]:
%%time
ds_out = regridder(ds_subset)
print(ds_out)

In [ ]:
da = ds_out.Tair.sel(time='2019-03-25 00:00', method='nearest').unify_chunks()
da.hvplot.quadmesh(x='lon', y='lat', geo=True, rasterize=True, cmap='turbo', tiles='OSM')

In [ ]:
%%time
ds_out.load().to_netcdf('HRRR_de_bay_rectilinear.nc', engine='h5netcdf', mode='w')